# Create Transects and Polygons from Centerline

Note: The user input centerline shape file must have a Coordinate Reference System (CRS) that is projected as opposed to geographic. A CRS is also known as a Spatial Reference System (SRS). Examples of a projected CRS are a state plane or Universal Transverse Mercator (UTM).


## Initialize

In [1]:
#jupyter nbextension enable --py --sys-prefix ipyleaflet
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import ipywidgets as widgets
from IPython.display import display, display_html
from ipyfilechooser import FileChooser
import functools
from traitlets import traitlets


## User Input Widgets & Select File Widget

In [2]:
# Define User Input Widgets
lbl1 = widgets.HTML(
    value= '<b>Number of CE-QUAL-W2 Segments)</b>',
)

seg_no_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000000,
    step=1,
    description='Segments:',
    disabled=False
)

lbl2 = widgets.HTML(
    value= '<b>Transect Width (units are the same as the CRS associated with the centerline shape file)</b>',
)

width_input = widgets.FloatText(
    value=0,
    description='Width:',
    disabled=False
)

lbl3 = widgets.HTML(
    value= '<b>Please enter local or mapped drive letter below where centerline shape file is located:</b>',
)

drive_letter = widgets.Text(
    value='C',
    placeholder='enter drive letter here',
    description='Drive Letter:',
    disabled=False
)

# Display Widgets
display(lbl1)
display(seg_no_input)
display(lbl2)
display(width_input)
display(lbl3)
display(drive_letter)

HTML(value='<b>Number of CE-QUAL-W2 Segments)</b>')

BoundedIntText(value=1, description='Segments:', max=1000000, min=1)

HTML(value='<b>Transect Width (units are the same as the CRS associated with the centerline shape file)</b>')

FloatText(value=0.0, description='Width:')

HTML(value='<b>Please enter local or mapped drive letter below where centerline shape file is located:</b>')

Text(value='C', description='Drive Letter:', placeholder='enter drive letter here')

In [14]:
#Add colon after drive letter
drive_letter_str = drive_letter.value + r":"


#### Code from: "https://pypi.org/project/ipyfilechooser/" ####

# Create and display a FileChooser widget
fc = FileChooser(drive_letter_str)
fc.filter_pattern = '*.shp'
display(fc)
    
# Change defaults and reset the dialog
fc.default_path = drive_letter_str
fc.default_filename = 'example_CL.shp'
fc.reset()
    
# Shorthand reset
fc.reset(path=drive_letter_str, filename='example_CL.shp')
    
# Change hidden files
fc.show_hidden = True
    
# Show or hide folder icons
fc.use_dir_icons = True
    
# Switch to folder-only mode
fc.show_only_dirs = False
    
# Set a file filter pattern (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = '*.shp'
    
# Change the title (use '' to hide)
fc.title = '<b>Select Centerline Shape File</b>'

# Sample callback function
def change_title(chooser):
    chooser.title = '<b>Select Centerline Shape File</b>'

# Register callback function
fc.register_callback(change_title)

FileChooser(path='I:', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False'…

In [4]:
#Number of CE-QUAL-W2 Segments (USER INPUT)
Seg_No = seg_no_input.value

#Cross Section Width
Width = width_input.value  #(USER INPUT) [Units are the same as the projected CRS provided in the centerline shape file]

#Import shape file centerline (USER INPUT)
wrkDir = fc.selected_path
pth_CL = fc.selected
geo_df_CL = gpd.read_file(pth_CL)
geo_df_CL_cntrd = gpd.read_file(pth_CL)
geo_df_CL_bndry = gpd.read_file(pth_CL)
CL_crs = geo_df_CL.crs

## Find Vertices from Centerline

In [5]:
geo_df_CL['points'] = geo_df_CL.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
CL_list = geo_df_CL['points'].tolist()
CL_array = np.array(CL_list)
CL_array_x = CL_array[0,...,0]
CL_array_y = CL_array[0,...,1]

## Find total length of the input Centerline

In [6]:
n = 0
dist_total = 0

for row in CL_array_x:
    if n == 0:
        x_pre = CL_array_x[n]
        y_pre = CL_array_y[n]
        n = n + 1
    else:
        x_n = CL_array_x[n]
        y_n = CL_array_y[n]
        dist_seg = np.sqrt((x_n - x_pre)**2 + (y_n - y_pre)**2)
        dist_total = dist_total + dist_seg
        x_pre = x_n
        y_pre = y_n
        n = n + 1
        CL_total_vertices = n

## Find W2 transect mid-points and centerline segment bounding vertices along input centerline

In [7]:
Seg_Lngth = dist_total / Seg_No
n = 0
k = 0
Seg_total = 0
CLpts_W2seg_total = Seg_No + 1
CLpts_W2seg_shape = (CLpts_W2seg_total, 1)
CLpts_W2seg_Dist = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_X = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_Y = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_bnd_X1 = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_bnd_X2 = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_bnd_Y1 = np.zeros(CLpts_W2seg_shape)
CLpts_W2seg_bnd_Y2 = np.zeros(CLpts_W2seg_shape)

for row in CLpts_W2seg_Dist: 
    if k == 0:  #this if condition populates the data for the first point along the centerline
        x_pre = CL_array_x[n]
        y_pre = CL_array_y[n]
        x_n = CL_array_x[n + 1]
        y_n = CL_array_y[n + 1]
        np.put(CLpts_W2seg_Dist, [k], [0])
        np.put(CLpts_W2seg_X, [k], [x_pre])
        np.put(CLpts_W2seg_Y, [k], [y_pre])
        np.put(CLpts_W2seg_bnd_X1, [k], [x_pre])
        np.put(CLpts_W2seg_bnd_X2, [k], [x_n])
        np.put(CLpts_W2seg_bnd_Y1, [k], [y_pre])
        np.put(CLpts_W2seg_bnd_Y2, [k], [y_n])
        k = k + 1
    elif k == (CLpts_W2seg_total - 1):  #this if condition populates the data for the last point along the centerline
        x_pre = CL_array_x[CL_total_vertices - 2]
        y_pre = CL_array_y[CL_total_vertices - 2]
        x_n = CL_array_x[CL_total_vertices - 1]
        y_n = CL_array_y[CL_total_vertices - 1]        
        np.put(CLpts_W2seg_Dist, [k], [dist_total])
        np.put(CLpts_W2seg_X, [k], [x_n])
        np.put(CLpts_W2seg_Y, [k], [y_n])
        np.put(CLpts_W2seg_bnd_X1, [k], [x_pre])
        np.put(CLpts_W2seg_bnd_X2, [k], [x_n])
        np.put(CLpts_W2seg_bnd_Y1, [k], [y_pre])
        np.put(CLpts_W2seg_bnd_Y2, [k], [y_n])
    else:
        running_total = 0
        Seg_total = Seg_total + Seg_Lngth
        np.put(CLpts_W2seg_Dist, [k], [Seg_total])
        n = 0
        x_pre = CL_array_x[n]
        y_pre = CL_array_y[n]
        n = n + 1
        while  Seg_total >= running_total:  #this while statement determines if the centerline segment has a W2 transect mid-point along it
            x_n = CL_array_x[n]
            y_n = CL_array_y[n]
            dist_seg = np.sqrt((x_n - x_pre)**2 + (y_n - y_pre)**2)
            running_total = running_total + dist_seg
            np.put(CLpts_W2seg_bnd_X1, [k], [x_pre])
            np.put(CLpts_W2seg_bnd_X2, [k], [x_n])
            np.put(CLpts_W2seg_bnd_Y1, [k], [y_pre])
            np.put(CLpts_W2seg_bnd_Y2, [k], [y_n])    
            x_pre = x_n
            y_pre = y_n
            n = n + 1    
        running_total_pre = running_total - dist_seg
        D = Seg_total - running_total_pre
        x_pre = CL_array_x[n - 2]
        y_pre = CL_array_y[n - 2]
        dx_sign = np.sign(x_n - x_pre)
        if dx_sign == 0: #this if statement traps the undefined slope of the CL segment
            #dx = 0 #x-coor is the same as x_pre since the CL segment is vertical
            dy_sign = np.sign(y_n - y_pre)
            dy = dy_sign * D
            CL_PtX = x_pre
            CL_PtY = y_pre + dy
        else:
            m_seg = (y_n - y_pre)/(x_n - x_pre)
            b = y_pre - (m_seg * x_pre)
            dx = D/(np.sqrt(1 + m_seg**2))
            dx = dx_sign * dx
            CL_PtX = x_pre + dx
            CL_PtY = m_seg * CL_PtX + b
            
        np.put(CLpts_W2seg_X, [k], [CL_PtX])
        np.put(CLpts_W2seg_Y, [k], [CL_PtY])
        k = k + 1

## Find End Points in the Transect

In [8]:
n = 0
D = Width/2

CLpts_W2seg_m_seg = np.zeros((CLpts_W2seg_Y.shape))
CLpts_W2seg_m = np.zeros((CLpts_W2seg_Y.shape))
CLpts_W2seg_PtX2_pos = np.zeros((CLpts_W2seg_Y.shape))
CLpts_W2seg_PtY2_pos = np.zeros((CLpts_W2seg_Y.shape))
CLpts_W2seg_PtX2_neg = np.zeros((CLpts_W2seg_Y.shape))
CLpts_W2seg_PtY2_neg = np.zeros((CLpts_W2seg_Y.shape))
Width_Check_array = np.zeros((CLpts_W2seg_Y.shape))


for row in CLpts_W2seg_X:
    SegX1 = CLpts_W2seg_bnd_X1[n]
    SegX2 = CLpts_W2seg_bnd_X2[n]
    SegY1 = CLpts_W2seg_bnd_Y1[n]
    SegY2 = CLpts_W2seg_bnd_Y2[n]
    PtX1 = CLpts_W2seg_X[n] #This is the same value as row
    PtY1 = CLpts_W2seg_Y[n]
    m_seg = (SegY2 - SegY1)/(SegX2 - SegX1) #Need an if statement here to trap errors with zero & undefined slopes
    m = -1 / m_seg    #This is the slope perpendicular to the CenterLine Segment
    b = PtY1 - (m * PtX1)
    dX = D/(np.sqrt(1 + m**2))
    PtX2_pos = PtX1 + dX
    PtX2_neg = PtX1 - dX
    PtY2_pos = m * PtX2_pos + b
    PtY2_neg = m * PtX2_neg + b
    Width_Check = np.sqrt((PtX2_pos - PtX2_neg)**2 + (PtY2_pos - PtY2_neg)**2)
    dx_seg = SegX2 - SegX1
    dy_seg = SegY2 - SegY1
    dx_Ptpos = PtX2_pos - PtX1
    dy_Ptpos = PtY2_pos - PtY1
    dx_Ptneg = PtX2_neg - PtX1
    dy_Ptneg = PtY2_neg - PtY1    
    crss_pos = (dx_seg * dy_Ptpos) - (dy_seg * dx_Ptpos)
    crss_neg = (dx_seg * dy_Ptneg) - (dy_seg * dx_Ptneg)
    crss_pos_sgn = np.sign(crss_pos)
    crss_neg_sgn = np.sign(crss_neg)
    
    if crss_pos_sgn == 1:
        PtX2_pos_crss = PtX2_pos
    else: PtX2_neg_crss = PtX2_pos
    
    if crss_pos_sgn == 1:
        PtY2_pos_crss = PtY2_pos
    else: PtY2_neg_crss = PtY2_pos    
    
    if crss_neg_sgn == 1:
        PtX2_pos_crss = PtX2_neg
    else: PtX2_neg_crss = PtX2_neg   
    
    if crss_neg_sgn == 1:
        PtY2_pos_crss = PtY2_neg
    else: PtY2_neg_crss = PtY2_neg    
    
    np.put(CLpts_W2seg_m_seg, [n], [m_seg])
    np.put(CLpts_W2seg_m, [n], [m])
    np.put(CLpts_W2seg_PtX2_pos, [n], [PtX2_pos_crss])
    np.put(CLpts_W2seg_PtY2_pos, [n], [PtY2_pos_crss])
    np.put(CLpts_W2seg_PtX2_neg, [n], [PtX2_neg_crss])
    np.put(CLpts_W2seg_PtY2_neg, [n], [PtY2_neg_crss])
    np.put(Width_Check_array, [n], [Width_Check])
    n = n + 1

C:\Users\jrutyna\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


## Create Geopanda Data Frame for Transect Lines

In [9]:
#Create transect end point array
Transect_Pts_A = np.column_stack((CLpts_W2seg_Dist, CLpts_W2seg_PtX2_pos, CLpts_W2seg_PtY2_pos))
Transect_Pts_B = np.column_stack((CLpts_W2seg_Dist, CLpts_W2seg_PtX2_neg, CLpts_W2seg_PtY2_neg))
Transect_Pts = np.concatenate((Transect_Pts_A, Transect_Pts_B), axis=0)

#Create pandas data frame for transect end points
df_transects = pd.DataFrame(data=Transect_Pts, columns=["CL_Distance", "Transect_Xcoor", "Transect_Ycoor"])

#Zip the coordinates into a point object and convert to a geopanda data frame
geometry = [Point(xy) for xy in zip(df_transects.Transect_Xcoor, df_transects.Transect_Ycoor)]
geo_df_transect_pts = gpd.GeoDataFrame(df_transects, geometry=geometry)

#Create geopanda data frame for transect lines
geo_df_transect_lines = geo_df_transect_pts.groupby(['CL_Distance'])['geometry'].apply(lambda x: LineString(x.tolist()))
geo_df_transect_lines = gpd.GeoDataFrame(geo_df_transect_lines, geometry='geometry', crs = CL_crs)


## Create and populate CE-QUAL-W2 polygons point arrays

In [10]:
n = 1 #starting "for loop" at second record of the "CLpts_W2seg_Pts#2_***" arrays to facilitate calling previous transect points
CLpts_W2segPlygn_shape = (Seg_No, 1)
W2seg_Plygn_Pt1_X = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt2_X = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt3_X = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt4_X = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt1_Y = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt2_Y = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt3_Y = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_Pt4_Y = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_CLdist = np.zeros((CLpts_W2segPlygn_shape))
W2seg_Plygn_ID = np.zeros((CLpts_W2segPlygn_shape))

for row in W2seg_Plygn_Pt1_X:
    Pt1_X = CLpts_W2seg_PtX2_neg[n]
    Pt1_Y = CLpts_W2seg_PtY2_neg[n]
    Pt2_X = CLpts_W2seg_PtX2_pos[n]
    Pt2_Y = CLpts_W2seg_PtY2_pos[n]
    Pt3_X = CLpts_W2seg_PtX2_pos[n - 1]
    Pt3_Y = CLpts_W2seg_PtY2_pos[n - 1]
    Pt4_X = CLpts_W2seg_PtX2_neg[n - 1]
    Pt4_Y = CLpts_W2seg_PtY2_neg[n - 1]
    CLdist = CLpts_W2seg_Dist[n]
    Plygn_ID = n
    np.put(W2seg_Plygn_Pt1_X , [n - 1], [Pt1_X])
    np.put(W2seg_Plygn_Pt2_X , [n - 1], [Pt2_X])
    np.put(W2seg_Plygn_Pt3_X , [n - 1], [Pt3_X])
    np.put(W2seg_Plygn_Pt4_X , [n - 1], [Pt4_X])
    np.put(W2seg_Plygn_Pt1_Y , [n - 1], [Pt1_Y])
    np.put(W2seg_Plygn_Pt2_Y , [n - 1], [Pt2_Y])
    np.put(W2seg_Plygn_Pt3_Y , [n - 1], [Pt3_Y])
    np.put(W2seg_Plygn_Pt4_Y , [n - 1], [Pt4_Y])
    np.put(W2seg_Plygn_CLdist , [n - 1], [CLdist])
    np.put(W2seg_Plygn_ID , [n - 1], [Plygn_ID])
    n = n + 1

## Create Geopanda Data Frame for W2 Polygon Segments

In [11]:
#Assemble polygon points for use in pandas
Polygon_Pts_A = np.column_stack((W2seg_Plygn_ID, W2seg_Plygn_CLdist, W2seg_Plygn_Pt1_X, W2seg_Plygn_Pt1_Y))
Polygon_Pts_B = np.column_stack((W2seg_Plygn_ID, W2seg_Plygn_CLdist, W2seg_Plygn_Pt2_X, W2seg_Plygn_Pt2_Y))
Polygon_Pts_C = np.column_stack((W2seg_Plygn_ID, W2seg_Plygn_CLdist, W2seg_Plygn_Pt3_X, W2seg_Plygn_Pt3_Y))
Polygon_Pts_D = np.column_stack((W2seg_Plygn_ID, W2seg_Plygn_CLdist, W2seg_Plygn_Pt4_X, W2seg_Plygn_Pt4_Y))
Polygon_Pts = np.concatenate((Polygon_Pts_A, Polygon_Pts_B, Polygon_Pts_C, Polygon_Pts_D), axis=0)

#Create pandas data frame for W2 polygon segments
df_polygon = pd.DataFrame(data=Polygon_Pts, columns=["W2segID", "CL_Distance", "Plygn_Xcoor", "Plygn_Ycoor"])

#Zip the coordinates into a point object and convert to a geopanda data frame
geometry = [Point(xy) for xy in zip(df_polygon.Plygn_Xcoor, df_polygon.Plygn_Ycoor)]
geo_df_polygon_pts = gpd.GeoDataFrame(df_polygon, geometry=geometry)

#Create geopanda data frame for W2 polygon segments
geo_df_polygon = geo_df_polygon_pts.groupby(['W2segID'])['geometry'].apply(lambda x: Polygon(x.tolist()))
geo_df_polygon = gpd.GeoDataFrame(geo_df_polygon, geometry='geometry', crs = CL_crs)

# Map Centerline, W2 Segment Transects, & W2 Segment Polygons)

## Initialize for Mapping

In [12]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap, SearchControl,  FullScreenControl, ScaleControl, 
 MeasureControl)
from ipywidgets import Text, HTML
from branca.colormap import linear
import json

## Create Map Layers, Add Map Controls, and Display Map

In [13]:
#Provide initial center point
geo_df_CL_cntrd['CL_centroid'] = geo_df_CL_cntrd.centroid
geo_df_CL_cntrd = geo_df_CL_cntrd.set_geometry('CL_centroid')
geo_df_CL_cntrd_mapCRS = geo_df_CL_cntrd.to_crs(4326)
geo_df_CL_cntrd_mapCRS['points'] = geo_df_CL_cntrd_mapCRS.apply(lambda x: [y for y in x['CL_centroid'].coords], axis=1)
cntrd_list = geo_df_CL_cntrd_mapCRS['points'].tolist()
cntrd_array = np.array(cntrd_list)
cntrd_array_y = cntrd_array[0,...,0]
cntrd_array_x = cntrd_array[0,...,1]
center = [cntrd_array_x[0], cntrd_array_y[0]]

#Provide initial zoom level and find bounds of centerline for "fit_bounds" map method
zoom = 6
geo_df_CL_bndry['bndry'] = geo_df_CL_bndry.to_crs(4326).boundary
bndry_list = geo_df_CL_bndry['bndry'].tolist()
bndry_array = np.array(bndry_list)
bndry_array_y = bndry_array[0,...,0]
bndry_array_x = bndry_array[0,...,1]
map_bounds = ((bndry_array_x[1], bndry_array_y[1]), (bndry_array_x[0], bndry_array_y[0]))

#Create map and provide basemap
m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m.fit_bounds(map_bounds)

#Create and add spatial data layers to the map
map_CL = GeoData(geo_dataframe = geo_df_CL.to_crs(4326),
                   style={'color': 'yellow', 'opacity':3, 'weight':1.9, 'dashArray':'0', 'fillOpacity':1},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'Centerline')

map_transects = GeoData(geo_dataframe = geo_df_transect_lines.to_crs(4326),
                   style={'color': 'red', 'opacity':3, 'weight':1.9, 'dashArray':'0', 'fillOpacity':1},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
                   name = 'W2 Transects')

map_polygons = GeoData(geo_dataframe = geo_df_polygon.to_crs(4326),
                   style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'green' , 'fillOpacity': 0.2},
                   name = 'W2 Segments')

m.add_layer(map_polygons)
m.add_layer(map_transects)
m.add_layer(map_CL)


#Add map controls to the map
m.add_control(LayersControl())
m.add_control(FullScreenControl())
m.add_control(ScaleControl(position='bottomleft'))

measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'meters'
)
m.add_control(measure)

measure.completed_color = 'red'

measure.add_length_unit('feet', 3.28084, 4)
measure.secondary_length_unit = 'feet'

measure.add_area_unit('sqfeet', 10.7639, 4)
measure.secondary_area_unit = 'sqfeet'


#Display the map
print("NOTE: map can be zoomed to a rectangular area specified by dragging the mouse while pressing the shift key")
m


NOTE: map can be zoomed to a rectangular area specified by dragging the mouse while pressing the shift key


Map(center=[38.291528915017324, -122.31211118815617], controls=(ZoomControl(options=['position', 'zoom_in_text…